<a href="https://colab.research.google.com/github/Koftarik/Data_Processing_and_Machine_Learning/blob/main/2_1_moving_objects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Необходимо разработать программу, детектирующую движение в видеопотоке. Требуется получить доступ к видеопотоку и получать очередные кадры из него. Необходимо получить зоны движения в видеопотоке и корректно определить тип движения в потоке. Написанная работоспособная программа должна обрабатывать не менее двух кадров в секунду.


In [ ]:

import numpy as np
import cv2
from matplotlib import pyplot as plt

In [ ]:
cap = cv2.VideoCapture("video.mp4"); #вывод кадров из видео файла
#cap = cv2.VideoCapture(0); # видео поток с веб камеры

cap.set(3,1280) # установка размера окна
cap.set(4,700)

ret, frame1 = cap.read()
ret, frame2 = cap.read()

output_video_path = 'output.avi'
output_video_res = (1280, 720)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_movie = cv2.VideoWriter(output_video_path, fourcc, 30.0, output_video_res)

while cap.isOpened(): # метод isOpened() выводит статус видеопотока
  if (frame1 is None) or (frame2 is None):
    break
  diff = cv2.absdiff(frame1, frame2) # нахождение разницы двух кадров, которая проявляется лишь при изменении одного из них, т.е. с этого момента наша программа реагирует на любое движение.

  gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # перевод кадров в черно-белую градацию

  blur = cv2.GaussianBlur(gray, (5, 5), 0) # фильтрация лишних контуров

  _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY) # метод для выделения кромки объекта белым цветом

  dilated = cv2.dilate(thresh, None, iterations = 3) # данный метод противоположен методу erosion(), т.е. эрозии объекта, и расширяет выделенную на предыдущем этапе область


  сontours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # нахождение массива контурных точек


  for contour in сontours:
    (x, y, w, h) = cv2.boundingRect(contour) # преобразование массива из предыдущего этапа в кортеж из четырех координат

    # метод contourArea() по заданным contour точкам, здесь кортежу, вычисляет площадь зафиксированного объекта в каждый момент времени, это можно проверить
    print(cv2.contourArea(contour))

    if cv2.contourArea(contour) < 700: # условие при котором площадь выделенного объекта меньше 700 px
      continue
    cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2) # получение прямоугольника из точек кортежа
    cv2.putText(frame1, "Status: {}".format("Dvigenie"), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA) # вставляем текст

  #cv2.drawContours(frame1, сontours, -1, (0, 255, 0), 2) также можно было просто нарисовать контур объекта

 # cv2.imshow("frame1", frame1)
  frame1 = frame2  #
  ret, frame2 = cap.read() #

  #if cv2.waitKey(40) == 27:
  #  break




cap.release()
cv2.destroyAllWindows()

48.5
48.0
69.0
60.0
49.0
684.0
36.0
55.0
48.5
56.0
2024.0
187.5
152.5
134.5
75.0
2682.5
1594.5
62.5
49.0
42.0
36.0
48.5
44858.5
11.5
7.0
2.0
2.0
2.0
2.0
2.0
2110.5
2.0
5.5
31.5
27.0
2.0
42.0
48.0


In [ ]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

def get_background(file_path):
  cap = cv2.VideoCapture(file_path)
  frame_indices = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=50)
  frames = []
  for idx in frame_indices:
    cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
  ret, frame = cap.read()
  frames.append(frame)
  median_frame = np.median(frames, axis=0).astype(np.uint8)
  return median_frame

cap = cv2.VideoCapture('video.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
save = 'video_2.mp4'
video_out = cv2.VideoWriter(
 save,
 cv2.VideoWriter_fourcc(*'mp4v'), 10,
 (frame_width, frame_height)
)

background = get_background('video.mp4')
background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
frame_count = 0
consecutive_frame = 4

while (cap.isOpened()):
 ret, frame = cap.read()
 if ret == True:
  frame_count += 1
  orig_frame = frame.copy()
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 if frame_count % consecutive_frame == 0 or frame_count == 1:
  frame_diff_list = []
  frame_diff = cv2.absdiff(gray, background)
  ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
  dilate_frame = cv2.dilate(thres, None, iterations=2)
  frame_diff_list.append(dilate_frame)
 if len(frame_diff_list) == consecutive_frame:
  sum_frames = sum(frame_diff_list)
  contours, hierarchy = cv2.findContours(sum_frames, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
 for i, cnt in enumerate(contours):
  cv2.drawContours(frame, contours, i, (0, 0, 255), -1)
 for contour in contours:
  if cv2.contourArea(contour) < 500:
    continue
  (x, y, w, h) = cv2.boundingRect(contour)
  cv2.rectangle(orig_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
  cv2_imshow(orig_frame)
  video_out.write(orig_frame)
else:
  break
video_out.release()
cap.release()
cv2.destroyAllWindows()

SyntaxError: ignored

In [ ]:
import cv2

from dotenv import load_dotenv
from vidstab import VidStab

if __name__ == "__main__":
    import os
    import cv2
    from vidstab import VidStab, layer_overlay, download_ostrich_video

    # Download test video to stabilize
    # if not os.path.isfile("ostrich.mp4"):
    #     download_ostrich_video("ostrich.mp4")

    # инициализируем трекер объектов, стабилизатор и захват потока
    object_tracker = cv2.TrackerCSRT_create()
    stabilizer = VidStab()
    vidcap = cv2.VideoCapture(0)

    # инициализируем форму для захвата объекта
    object_bounding_box = None

    while True:
        # читаем фрейм
        grabbed_frame, frame = vidcap.read()

        # передаем фрейм в стабилизатор
        stabilized_frame = stabilizer.stabilize_frame(input_frame=frame, border_size=50, smoothing_window=100)

        if stabilized_frame is None:
            break

        # рисуем прямоугольник на выделенном объекте, если начали трекинг
        if object_bounding_box is not None:
            success, object_bounding_box = object_tracker.update(stabilized_frame)

            if success:
                (x, y, w, h) = [int(v) for v in object_bounding_box]
                cv2.rectangle(stabilized_frame, (x, y), (x + w, y + h),
                              (0, 255, 0), 2)

        # выводим фрейм
        cv2.imshow('Frame', stabilized_frame)

        key = cv2.waitKey(5)

        # захватываем фрейм
        if stabilized_frame.sum() > 0 and object_bounding_box is None:
            object_bounding_box = cv2.selectROI("Frame",
                                                stabilized_frame,
                                                fromCenter=False,
                                                showCrosshair=True)
            object_tracker.init(stabilized_frame, object_bounding_box)
        elif key == 27:
            break

    vidcap.release()
    cv2.destroyAllWindows()

In [ ]:
import cv2 # импорт модуля cv2

cap = cv2.VideoCapture("video.mp4"); #вывод кадров из видео файла
#cap = cv2.VideoCapture(0); # видео поток с веб камеры

cap.set(3,1280) # установка размера окна
cap.set(4,700)

ret, frame1 = cap.read()
ret, frame2 = cap.read()

while cap.isOpened(): # метод isOpened() выводит статус видеопотока

  diff = cv2.absdiff(frame1, frame2) # нахождение разницы двух кадров, которая проявляется лишь при изменении одного из них, т.е. с этого момента наша программа реагирует на любое движение.

  gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) # перевод кадров в черно-белую градацию

  blur = cv2.GaussianBlur(gray, (5, 5), 0) # фильтрация лишних контуров

  _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY) # метод для выделения кромки объекта белым цветом

  dilated = cv2.dilate(thresh, None, iterations = 3) # данный метод противоположен методу erosion(), т.е. эрозии объекта, и расширяет выделенную на предыдущем этапе область


  сontours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # нахождение массива контурных точек


  for contour in сontours:
    (x, y, w, h) = cv2.boundingRect(contour) # преобразование массива из предыдущего этапа в кортеж из четырех координат

    # метод contourArea() по заданным contour точкам, здесь кортежу, вычисляет площадь зафиксированного объекта в каждый момент времени, это можно проверить
    print(cv2.contourArea(contour))

    if cv2.contourArea(contour) < 700: # условие при котором площадь выделенного объекта меньше 700 px
      continue
    cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2) # получение прямоугольника из точек кортежа
    cv2.putText(frame1, "Status: {}".format("Dvigenie"), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3, cv2.LINE_AA) # вставляем текст

  #cv2.drawContours(frame1, сontours, -1, (0, 255, 0), 2) также можно было просто нарисовать контур объекта

  cv2.imshow("frame1", frame1)
  frame1 = frame2  #
  ret, frame2 = cap.read() #

  if cv2.waitKey(40) == 27:
    break




cap.release()
cv2.destroyAllWindows()

48.5
48.0
69.0
60.0
49.0
684.0
36.0
55.0
48.5
56.0
94.0
106.0
121.0
48.0
74.0
55.5
86.0
102.5
134.5
88.5
62.5
78.5
77.5
49.0
54327.5
2.0
2.0
2.0
2.0
16.0
2.0
2.0
57.5
19.0
539.5
25.0
1027.5
112.5
82.5
67.5
216.0
3633.5
11.5
140.0
8.5
2.0
2.0
36.0
98.5
29890.0
27.0
4.0
2.0
2.0
2.0
42.0
36.0
48.0
42.0
36.0
42.0
42.0


DisabledFunctionError: ignored